# Validation - BAC response of models

This notebook evaluates qualitatively the capability of fitted models to reproduce BAC behavior.

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path
import pickle
import pandas as pd

import matplotlib.pyplot as plt
import neuroplotlib as nplt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import LFPy
import neuron
import numpy as np

import shutil

import model
import evaluator
import plotting
import utils

# %matplotlib notebook
%matplotlib widget

In [ ]:
probe_type = "planar"
model_name = "hay" #'hallermann' # "hay" hay_ais
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
data_folder = Path('..') / "data" / f"{model_name}_ecode_probe_{probe_type}"
result_folder = Path("..") / "results" / 'models' / 'opt_results' 

In [ ]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".").absolute()
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")
    compiled_folder = Path("./x86_64")
    if compiled_folder.is_dir():
        shutil.rmtree(compiled_folder)
    shutil.copytree(model_folder / "x86_64", compiled_folder)

In [ ]:
data = pickle.load(open(result_folder / 'runs.pkl', 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

In [ ]:
opt_soma = df_model.query("feature_set == 'soma'")
opt_extra = df_model.query("feature_set == 'extra'")
print(f"Somatic optimizations: {len(opt_soma)}")
print(f"Extra optimizations: {len(opt_extra)}")

In [ ]:
protocols_file = data_folder / f"{model_name}_ecode_probe_{probe_type}/efeatures/protocols_BPO.json"
features_file = data_folder /  f"{model_name}_ecode_probe_{probe_type}/efeatures/features_BPO.json"

In [ ]:
ais_recordings = {}

In [ ]:
cell = model.create(model_name=model_name)
cell_release = model.create(model_name=model_name, release=True)

probe = model.define_electrode(probe_type=probe_type)
# probe=None

param_names = [param.name for param in cell.params.values() if not param.frozen]
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

In [ ]:
params_release

# Test calcium spike

In [ ]:
def simulate_BAC_responses(cell, params, sim, pulse_delay=10, pulse_amp=1, pulse_dur=5,
                           syn_tau=2, syn_weight=5, syn_delay=13, dend_y_stim=620,
                           dend_y_rec=[400, 620]):
    # instantiate
    cell.freeze(param_dict=params)
    cell.instantiate(sim=sim)
    lfpy_cell = cell.LFPyCell
    
    # define args for stimuli
    soma_args = {
         'idx': 0,
         'record_current': True,
         'pptype': 'IClamp',
         'amp': pulse_amp,
         'dur': pulse_dur,
         'delay': pulse_delay
    }

    stim = LFPy.StimIntElectrode(cell=lfpy_cell, **soma_args)

    synapseParameters = {
         'idx': lfpy_cell.get_closest_idx(x=0, y=dend_y_stim, z=0),
         'e': 0,                               # reversal potential
         'syntype': 'ExpSyn',                  # synapse type
         'tau': syn_tau,                             # syn. time constant
         'weight': syn_weight,                          # syn. weight
         'record_current': True                # syn. current record
    }

    synapse = LFPy.Synapse(lfpy_cell, **synapseParameters)
    synapse.set_spike_times(np.array([syn_delay]))
    
    # simulate
    lfpy_cell.simulate(rec_vmem=True)
    
    # gather responses
    responses = {}
    responses["soma"] = {}
    responses["soma"]["time"] = lfpy_cell.tvec
    responses["soma"]["voltage"] = lfpy_cell.vmem[0]
    
    for d in dend_y_rec:
        responses[str(d)] = {}
        d_idx = lfpy_cell.get_closest_idx(x=0, y=d, z=0)
        resp = lfpy_cell.vmem[d_idx]
        responses[str(d)]["voltage"] = resp
        responses[str(d)]["time"] = lfpy_cell.tvec
        
    # destroy/unfreeze
    cell.destroy(sim=sim)
    cell.unfreeze(list(params.keys()))
    
    return responses

In [ ]:
plt.figure()
for resp_name, resp in resp_release.items():
    plt.plot(resp["time"], resp["voltage"], label=resp_name)
plt.legend()

In [ ]:
kwargs = {"syn_weight": 2}

In [ ]:
response_release = simulate_BAC_responses(cell, params_release, sim, **kwargs)

print("Computing SOMA")
responses_soma = []
for i, (index, opt) in enumerate(opt_soma.iterrows()):
    print(f"Computing {i + 1} / {len(opt_soma)}")
    params_dict = {k: v for k, v in zip(param_names, opt.best_params)}
    response = simulate_BAC_responses(cell, params_dict, sim, **kwargs)
    responses_soma.append(response)

print("Computing EXTRA")
responses_extra = []
for i, (index, opt) in enumerate(opt_extra.iterrows()):
    print(f"Computing {i + 1} / {len(opt_soma)}")
    params_dict = {k: v for k, v in zip(param_names, opt.best_params)}
    response = simulate_BAC_responses(cell, params_dict, sim, **kwargs)
    responses_extra.append(response)

In [ ]:
ax_dict = {}

loc_names = resp_release.keys()

for loc in loc_names:
    fig, ax = plt.subplots()
    ax_dict[loc] = ax

    for i, resp in enumerate(responses_soma):
        if i == 0:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C0", label="soma")
        else:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C0")

    for i, resp in enumerate(responses_extra):
        if i == 0:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C1", label="extra")
        else:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C1")

    ax.plot(resp_release[loc]["time"], resp_release[loc]["voltage"], alpha=0.8, color="k", lw=2, 
            label="GT")
    ax.set_title(f"Recording from {loc}")
    ax.legend()